# Final Evaluation
First we do the hyperparameter tuning for each encoder seperately using optuna. We using the TPESampler to do the tuning with 50 trials. In this file we using the best hyperparameter for each encoder to to train the model and do the cross validation using K-fold method.

In [1]:
%reload_ext autoreload
%autoreload all

%aimport -torch
%aimport -matplotlib
%aimport -seaborn
%aimport -numpy
%aimport -pandas
%aimport -scipy
%aimport -lightning 

## Load the data

In [2]:
from eeg_snn_encoder.config import MODELS_DIR, PROCESSED_DATA_DIR
from eeg_snn_encoder.dataset import CHBMITDataset

# Load the dataset
dataset = CHBMITDataset(PROCESSED_DATA_DIR / "stft_normalized.h5")

2025-05-09 14:07:57.614 | INFO     | eeg_snn_encoder.config:<module>:11 - PROJ_ROOT path is: /workspace/snn-encoder-test


In [3]:
import gc
import json
import os

from IPython.display import clear_output
import lightning.pytorch as pl
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from loguru import logger
import optuna
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler

from eeg_snn_encoder.config import REPORTS_DIR
from eeg_snn_encoder.models.classifier import ModelConfig
from eeg_snn_encoder.models.lightning import LitSeizureClassifier, OptimizerConfig

In [4]:
from eeg_snn_encoder.encoders import (
    BSAEncoder,
    BurstEncoder,
    DummyEncoder,
    PhaseEncoderExpand,
    PoissonEncoderExpand,
    StepForwardEncoder,
    TBREncoder,
)

## Prepare the config

In [ ]:
tuning_config = [
    {
        "codename": "be",
        "encoder": BurstEncoder,
        "encoder_params": lambda params: (
            {
                "max_window": params["max_window"],
                "n_max": params["n_max"],
                "t_max": params["t_max"],
                "t_min": params["t_min"],
            }
        ),
    },
    {
        "codename": "bsa",
        "encoder": BSAEncoder,
        "encoder_params": lambda params: (
            {
                "win_size": params["win_size"],
                "cutoff": params["cutoff"],
                "threshold": params["encoder_threshold"],
            }
        ),
    },
    {"codename": "dummy", "encoder": DummyEncoder, "encoder_params": lambda params: (dict())},
    {
        "codename": "pe",
        "encoder": PhaseEncoderExpand,
        "encoder_params": lambda params: (
            {
                "phase_window": params["phase_window"],
            }
        ),
    },
    {
        "codename": "poisson",
        "encoder": PoissonEncoderExpand,
        "encoder_params": lambda params: (
            {
                "interval_freq": 8,
                "random_seed": 47,
            }
        ),
    },
    {
        "codename": "sf",
        "encoder": StepForwardEncoder,
        "encoder_params": lambda params: (
            {
                "threshold": params["sf_threshold"],
            }
        ),
    },
    {
        "codename": "tbr",
        "encoder": TBREncoder,
        "encoder_params": lambda params: (
            {
                "threshold": params["tbr_threshold"],
            }
        ),
    },
]

## Load config and tuning
Load the best params from the intermediat value as optuna doesn't save the best value but latest one. And the tune the model and test it against the test set selected but K-Fold.

In [ ]:
for tuning in tuning_config:
    clear_output(wait=True)
    study_name = f"model-fine-tuning-{tuning['codename']}"
    study = optuna.load_study(
        study_name=study_name,
        storage=os.environ["OPTUNA_CONN_STRING"],
    )

    filtered_trials = [t for t in study.get_trials() if len(t.intermediate_values) > 35]

    best_trial = min(
        filtered_trials,
        key=lambda t: min(t.intermediate_values.values(), default=float("inf"))
    )

    best_params = best_trial.params

    model_params: ModelConfig = {
        "threshold": best_params["threshold"],
        "slope": best_params["slope"],
        "beta": best_params["beta"],
        "dropout_rate1": best_params["dropout_rate1"],
        "dropout_rate2": best_params["dropout_rate2"],
    }

    optimizer_params: OptimizerConfig = {
        "lr": best_params["lr"],
        "weight_decay": best_params["weight_decay"],
        "scheduler_factor": best_params["scheduler_factor"],
        "scheduler_patience": best_params["scheduler_patience"],
    }

    encoder_params = tuning["encoder_params"](best_params)

    logger.info(f"Starting {tuning['codename']}-encoder final evaluation")
    logger.info(
        f"Params model:{model_params}, optimizer:{optimizer_params}, encoder:{encoder_params}"
    )

    Encoder = tuning["encoder"]

    kfold = KFold(n_splits=5, shuffle=True, random_state=42)

    fold_results = []

    for fold, (train_val_ids, test_ids) in enumerate(kfold.split(dataset)):
        logger.info(f"Starting fold {fold + 1} of {kfold.n_splits} {tuning['codename']}-encoder")

        train_ids, val_ids = train_test_split(
            train_val_ids, test_size=0.2, random_state=42, shuffle=True
        )

        train_sampler = SubsetRandomSampler(train_ids)
        val_sampler = SubsetRandomSampler(val_ids)
        test_sampler = SubsetRandomSampler(test_ids)

        trainloader = DataLoader(dataset, batch_size=512, sampler=train_sampler)
        valloader = DataLoader(dataset, batch_size=512, sampler=val_sampler)
        testloader = DataLoader(dataset, batch_size=512, sampler=test_sampler)

        spike_encoder = Encoder(**encoder_params)

        lit_model = LitSeizureClassifier(
            model_config=model_params,
            optimizer_config=optimizer_params,
            spike_encoder=spike_encoder,
        )

        trainer = pl.Trainer(
            max_epochs=50,
            accelerator="auto",
            devices="auto",
            strategy="auto",
            default_root_dir=MODELS_DIR / f"{tuning['codename']}_encoder",
            callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=5)],
            logger=False,
        )

        trainer.fit(lit_model, trainloader, valloader)
        trainer.test(lit_model, testloader, ckpt_path="best")

        test_loss = trainer.callback_metrics["test_loss"].item()
        test_acc = trainer.callback_metrics["test_acc"].item()
        test_precision = trainer.callback_metrics["test_precision"].item()
        test_recall = trainer.callback_metrics["test_recall"].item()
        test_f1 = trainer.callback_metrics["test_f1"].item()
        test_mse = trainer.callback_metrics["test_mse"].item()
        test_total_spikes = trainer.callback_metrics["test_total_spikes"].item()

        logger.info(
            f"{tuning['codename']}-encoder ",
            f"Fold {fold + 1} - Test Loss: {test_loss:.4f}, "
            f"Test Accuracy: {test_acc:.4f}, "
            f"Test Precision: {test_precision:.4f}, "
            f"Test Recall: {test_recall:.4f}, "
            f"Test F1: {test_f1:.4f}, "
            f"Test MSE: {test_mse:.4f}, "
            f"Test Total Spikes: {test_total_spikes:.4f}",
        )

        fold_results.append(
            {
                "fold": fold,
                "test_loss": test_loss,
                "test_acc": test_acc,
                "test_precision": test_precision,
                "test_recall": test_recall,
                "test_f1": test_f1,
                "test_mse": test_mse,
                "test_total_spikes": test_total_spikes,
            }
        )

        del lit_model
        del trainer
        del spike_encoder
        torch.cuda.empty_cache()
        gc.collect()

    results_df = pd.DataFrame(fold_results)
    results_df.set_index("fold", inplace=True)

    results_df.to_csv(REPORTS_DIR / f"{tuning['codename']}_model_results.csv", index=True)

    params_file = REPORTS_DIR / f"{tuning['codename']}_model_params.json"

    params_file.write_text(
        json.dumps(
            {
                "model_params": model_params,
                "optimizer_params": optimizer_params,
                "encoder_params": encoder_params,
            },
            indent=4,
        )
    )
    logger.info(f"Results and parameters saved to {REPORTS_DIR} for {tuning['codename']}-encoder")

[I 2025-05-09 14:23:18,561] Using an existing study with name 'model-fine-tuning-tbr' instead of creating a new one.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                   | Params | Mode 
---------------------------------------------------------
0 | model | EEGSTFTSpikeClassifier | 824 K  | train
---------------------------------------------------------
824 K     Trainable params
0         Non-trainable params
824 K     Total params
3.299     Total estimated model params size (MB)
13        Modules in train mode
0         Modules in eval mode


2025-05-09 14:23:19.240 | INFO     | __main__:<module>:38 - Starting tbr-encoder final evaluation
2025-05-09 14:23:19.241 | INFO     | __main__:<module>:39 - Params model:{'threshold': 0.043919398650706404, 'slope': 10.50493804185783, 'beta': 0.8871877886982101, 'dropout_rate1': 0.42902441176790485, 'dropout_rate2': 0.10871506022527165}, optimizer:{'lr': 9.182800132489358e-05, 'weight_decay': 3.0314735054464033e-06, 'scheduler_factor': 0.2975191201261136, 'scheduler_patience': 10}, encoder:{'threshold': 0.957310794127068}
2025-05-09 14:23:19.242 | INFO     | __main__:<module>:48 - Starting fold 1 of 5 tbr-encoder


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Restoring states from the checkpoint path at /workspace/snn-encoder-test/models/tbr_encoder/checkpoints/epoch=27-step=196.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /workspace/snn-encoder-test/models/tbr_encoder/checkpoints/epoch=27-step=196.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7248255014419556     │
│          test_f1          │    0.7094742059707642     │
│         test_loss         │    12.335832595825195     │
│         test_mse          │    0.27517446875572205    │
│      test_precision       │     0.715631902217865     │
│        test_recall        │    0.7035815119743347     │
│     test_total_spikes     │        40707.34375        │
└───────────────────────────┴───────────────────────────┘

2025-05-09 14:36:17.814 | INFO     | __main__:<module>:91 - tbr-encoder 
2025-05-09 14:36:18.019 | INFO     | __main__:<module>:140 - Results and parameters saved to /workspace/snn-encoder-test/reports for tbr-encoder
